# Data

Inspect, clean and balance synthetic grasp samples.

In [ ]:
import os
os.chdir('..')

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from vgn.io import *
from vgn.perception import *
from vgn.utils.transform import Rotation, Transform

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
import sys
import rospy
from vgn import vis
rospy.init_node("vgn_vis", anonymous=True)

Path to the data folder.

In [10]:
root = Path("../data/raw/test")

## Inspection

Compute the number of positive and negative samples in the dataset.

In [11]:
df = read_df(root)

In [12]:
positives = df[df["label"] == 1]
negatives = df[df["label"] == 0]

print("Number of samples:", len(df.index))
print("Number of positives:", len(positives.index))
print("Number of negatives:", len(negatives.index))

Number of samples: 545
Number of positives: 74
Number of negatives: 471


Visualize a random sample. Make sure to have a ROS core running and open `config/sim.rviz` in RViz.

In [15]:
size, intrinsic, _, finger_depth = read_setup(root)

0.30000000000000004


In [14]:
i = np.random.randint(len(df.index))
scene_id, grasp, label = read_grasp(df, i)
depth_imgs, extrinsics = read_sensor_data(root, scene_id)

tsdf = create_tsdf(size, 120, depth_imgs, intrinsic, extrinsics)
tsdf_grid = tsdf.get_grid()
cloud = tsdf.get_cloud()

vis.clear()
vis.draw_workspace(size)
vis.draw_points(np.asarray(cloud.points))
vis.draw_grasp(grasp, label, finger_depth)

IndexError: index 40 is out of bounds for axis 2 with size 40

Plot the distribution of angles between the gravity vector and $Z$ axis of grasps.

In [ ]:
angles = np.empty(len(positives.index))
for i, index in enumerate(positives.index):
    approach = Rotation.from_quat(df.loc[index, "qx":"qw"].to_numpy()).as_matrix()[:,2]
    angle = np.arccos(np.dot(approach, np.r_[0.0, 0.0, -1.0]))
    angles[i] = np.rad2deg(angle)        

In [ ]:
plt.hist(angles, bins=30)
plt.xlabel("Angle [deg]")
plt.ylabel("Count")
plt.show()

## Cleanup

DANGER: the following lines will modify/delete data.

Remove grasp positions that lie outside the workspace.

In [ ]:
df = read_df(root)
df.drop(df[df["x"] < 0.02].index, inplace=True)
df.drop(df[df["y"] < 0.02].index, inplace=True)
df.drop(df[df["z"] < 0.02].index, inplace=True)
df.drop(df[df["x"] > 0.28].index, inplace=True)
df.drop(df[df["y"] > 0.28].index, inplace=True)
df.drop(df[df["z"] > 0.28].index, inplace=True)
write_df(df, root)

Remove unreferenced scenes.

In [ ]:
df = read_df(root)
scenes = df["scene_id"].values
for f in (root / "scenes").iterdir():
    if f.suffix == ".npz" and f.stem not in scenes:
        print("Removed", f)
        f.unlink()

## Balance

Discard a subset of negative samples to balance classes.

In [ ]:
df = read_df(root)

In [ ]:
positives = df[df["label"] == 1]
negatives = df[df["label"] == 0]
i = np.random.choice(negatives.index, len(negatives.index) - len(positives.index), replace=False)
df = df.drop(i)

In [ ]:
write_df(df, root)